# Validation

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.multimodal_har_model import MultiModalHARModel

Loading config...
Loading secrets...


2025-11-13 15:31:36,278 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-11-13 15:31:36,281 - INFO - Sentry initialized with environment: development


**Parameters**

In [2]:
MODEL_PHT_PATH = "/Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v1.0.0_nw_ucla_2025-11-11 09:45:21.993320.pht"
VALIDATION_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "validation"
)

## Loading Dataset

In [3]:
validation_video_data_loader = VideoDataLoader(
    path=VALIDATION_DIR,
)
validation_dataset = VideoDataset(
    video_data_loader=validation_video_data_loader,
    normalization_type="across_frames",
)

len(validation_dataset)
for _ in validation_dataset:
    pass


display(len(validation_dataset.labels_map))

2025-11-13 15:31:38,366 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-11-13 15:31:38,405 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-11-13 15:31:38,505 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-11-13 15:31:38,566 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-11-13 15:31:38,607 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-11-13 15:31:38,642 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-11-13 15:31:38,673 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-11-13 15:31:38,824 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-11-13 15:31:38,887 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-11-13 15:31:38,930 - INFO - [VideoDataLoader] Loding action videos for action: a12


10

## Loading Model

In [4]:
har_model, _ = MultiModalHARModel.load(
    checkpoint_path=MODEL_PHT_PATH
)

2025-11-13 15:31:39,452 - INFO - Loading model from /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v1.0.0_nw_ucla_2025-11-11 09:45:21.993320.pht...
2025-11-13 15:31:39,530 - INFO - Model config: {'obj_in': 5, 'joint_in': 3, 'gat_hidden': 128, 'gat_out': 128, 'temporal_hidden': 128, 'num_classes': 10, 'dropout': 0.1}
2025-11-13 15:31:39,531 - INFO - Model configuration: {'obj_in': 5, 'joint_in': 3, 'gat_hidden': 128, 'gat_out': 128, 'temporal_hidden': 128, 'num_classes': 10, 'dropout': 0.1}
2025-11-13 15:31:39,593 - INFO - ✅ Model loaded and ready for inference


## Quantitative Validation

- Accuracy
- AUC

In [5]:
from sklearn.metrics import roc_auc_score
import torch
from torch import nn
from typing import Tuple, List
from tqdm import tqdm

@torch.no_grad()
def evaluate_model(
    model: nn.Module,
    dataset: VideoDataset,
    device: torch.device
) -> Tuple[float, float]:
    """
    Evaluates a trained model on a VideoDataset.
    Computes accuracy and macro-averaged AUC.
    """
    model.eval()
    y_true_list: List[int] = []
    y_pred_logits_list: List[torch.Tensor] = []

    for video_data in tqdm(dataset, desc="Evaluating"):
        graphs_objects = [g.to(device) for g in video_data.graphs_objects]
        graphs_joints = [g.to(device) for g in video_data.graphs_joints]
        label = video_data.label.to(device)

        # Forward pass
        logits: torch.Tensor = model(graphs_objects, graphs_joints)
        y_pred_logits_list.append(logits.cpu())
        y_true_list.append(label.cpu().item())

    # Concatenate results
    y_true = torch.tensor(y_true_list)
    y_pred_logits = torch.cat(y_pred_logits_list, dim=0)

    # Compute metrics
    accuracy, auc = compute_metrics(y_true, y_pred_logits)
    return accuracy, auc


def compute_metrics(
    y_true: torch.Tensor,
    y_pred_logits: torch.Tensor
) -> Tuple[float, float]:
    """
    Compute accuracy and macro-AUC.
    Args:
        y_true: Tensor of shape (N,) with integer labels.
        y_pred_logits: Tensor of shape (N, C) with raw model outputs (before softmax).
    Returns:
        (accuracy, macro_auc)
    """
    y_true_np = y_true.numpy()
    y_pred_np = torch.softmax(y_pred_logits, dim=1).numpy()

    preds = y_pred_np.argmax(axis=1)
    accuracy: float = (preds == y_true_np).mean()

    try:
        auc: float = roc_auc_score(y_true_np, y_pred_np, multi_class='ovr', average='macro')
    except ValueError:
        auc = float('nan')

    return accuracy, auc


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
har_model.to(device)

accuracy, auc = evaluate_model(har_model, validation_dataset, device)
print(f"Validation Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")


Evaluating: 100%|██████████| 152/152 [00:02<00:00, 54.51it/s]

Validation Accuracy: 0.7368, AUC: 0.9731


## Qualitative Validation

- Graphic Analysis - Plotting with t-SNE